# Make a corpus of mini documents
The following code will create a corpus, which is the large subset of the entire BNC, that can be used with bag of words topic models.

For this, I assume you have downloaded either the *2554* BNC corpus or the *bnc_paragraphs.pkl* file. If you have the *bnc_paragraphs.pkl* file, you don't need the *2554* corpus, unless you want to re-create the *bnc_paragraphs.pkl*. 
These two files can be obtained by running, in a unix shell, the commands:
* getbnc2554.sh
* getbncparagraphs.sh

You'll also need some word lists, including stop word lists. These can be obtained with 
* getvocabularylists.sh

In [1]:
from collections import defaultdict

In [2]:
from bnctools import utils

In [3]:
use_cached_data = True

pkl_filename = 'bnc_paragraphs.pkl'
bnc_2554_texts_root = 'bnc/2554/download/Texts/'

if not use_cached_data:
    
    corpus_filenames = utils.Corpus.get_corpus_filenames(bnc_2554_texts_root)
    
    # Make sure cluster is started with e.g. ipcluster start -n 16
    view = utils.init_ipyparallel()
    paragraphs = utils.get_all_paragraphs_parallel(view, corpus_filenames)
    utils.dump(paragraphs, filename=pkl_filename)

else:
    
    paragraphs = utils.load(pkl_filename)

In [4]:
sum(map(lambda paragraph: paragraph['word_count'], paragraphs))

87564696

In [5]:
mini_documents = utils.paragraphs_to_mini_documents(paragraphs)

In [6]:
counts = map(lambda doc: len(doc.split('|')), mini_documents)

In [7]:
[f(counts) for f in (sum, min, max)]

[78723408, 250, 500]

In [8]:
with open('bnc_texts_%d_%d_%d.txt' % tuple([func(counts) for func in (sum, min, max)]), 'w') as f:
    f.write('\n'.join(mini_documents))

In [9]:
vocabulary = utils.get_corpus_vocabulary(mini_documents, minimum_count=5)

In [10]:
len(vocabulary)

49328

In [11]:
with open('bnc_vocab_%d.txt' % len(vocabulary), 'w') as f:
    f.write('\n'.join(sorted(vocabulary.keys())))